In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
import streamlit as st
import re

sw=pd.read_csv('https://raw.githubusercontent.com/Izainea/skill_matching/main/data/sw.csv')
sw=list(sw['vacias'])

DF=pd.read_csv('https://raw.githubusercontent.com/Izainea/skill_matching/main/data/datacompare.csv')
DF_dd=DF.loc[DF['NOMBRE DEL PROGRAMA '].drop_duplicates().index]
DF_con=pd.read_csv('https://raw.githubusercontent.com/Izainea/skill_matching/main/data/Consolidado_Cursos_Sena_2022.csv')
columns=DF_con.columns
DF_con['Nombre del curso']=DF_con['Nombre del curso (MINUSC)'].str.upper()+" "+DF_con['Palabras Clave'].str.upper()
DF_con['Nombre del curso']=DF_con['Nombre del curso'].str.upper().str.replace('TRABAJO ',' ').str.replace('SEGURO ',' ').str.replace(' EN ',' ').str.replace(' PARA ',' ').str.replace(' DE ',' ').str.replace(' Y ',' ').str.replace(' LAS ',' ').str.replace(' LOS ',' ').str.replace(' EL ',' ').str.replace(' LA ',' ').str.replace(' BASICO ',' ').str.replace(' OPERATIVO ',' ').str.replace('  ',' ')


from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def extract_best_indices(m, topk, mask=None):
    """
    Use sum of the cosine distance over all tokens ans return best mathes.
    m (np.array): cos matrix of shape (nb_in_tokens, nb_dict_tokens)
    topk (int): number of indices to return (from high to lowest in order)
    """
    # return the sum on all tokens of cosinus for each sentence
    if len(m.shape) > 1:
        cos_sim = np.mean(m, axis=0) 
    else: 
        cos_sim = m
    index = np.argsort(cos_sim)[::-1] # from highest idx to smallest score 
    if mask is not None:
        assert mask.shape == m.shape
        mask = mask[index]
    else:
        mask = np.ones(len(cos_sim))
    mask = np.logical_or(cos_sim[index] != 0, mask) #eliminate 0 cosine distance
    best_index = index[mask][:topk]
    dis=cos_sim[best_index]
    return best_index,dis


def get_recommendations_tfidf(sentence, tfidf_mat,n):
    
    """
    Return the database sentences in order of highest cosine similarity relatively to each 
    token of the target sentence. 
    """
    # Embed the query sentence
    tokens_query = [str(tok) for tok in sentence.split()]
    embed_query = vectorizer.transform(tokens_query)
    # Create list with similarity between query and dataset
    mat = cosine_similarity(embed_query, tfidf_mat)
    
    # Best cosine distance for each token independantly
    best_index,dis = extract_best_indices(mat, topk=n)
    
    return best_index,dis



# Fit TFIDF
vectorizer = TfidfVectorizer(stop_words=sw) 
tfidf_mat = vectorizer.fit_transform(DF_con['Nombre del curso'].fillna('NADA').str.upper()) # -> (num_sentences, num_vocabulary)
Lista=[]
Lista2=[]

def recomendador(j,n):
    try: 
        best_index,dis = get_recommendations_tfidf(j.upper(), tfidf_mat,n)
        dici=DF_con[columns].iloc[best_index].to_dict()
        if dis[0]>0.005:
            return dici
        else:
            return ["Nada que recomendar"]
        
    except:
        return ["Nada que recomendar"]

In [37]:
recomendador("Experto en alturas",10)

{'Proveedor': {172: 'SENA',
  174: 'SENA',
  171: 'SENA',
  27: 'SENA',
  170: 'SENA',
  159: 'SENA',
  88: 'SENA',
  54: 'SENA',
  61: 'SENA',
  60: 'SENA'},
 'Nombre del curso (MAYUSC)': {172: 'TRABAJO EN ALTURAS OPERATIVO',
  174: nan,
  171: 'TRABAJO EN ALTURAS AVANZADO',
  27: 'AVANZADO TRABAJO SEGURO EN ALTURAS',
  170: 'TRABAJO EN ALTURAS ADMINISTRATIVO',
  159: 'RE ENTRENAMIENTO NIVEL AVANZADO TRABAJO SEGURO EN ALTURAS',
  88: nan,
  54: nan,
  61: 'DISEÑO ASISTIDO POR COMPUTADOR - AUTOCAD 2D',
  60: 'DISEÑAR PÁGINAS WEB CON HTML Y JAVASCRIPT'},
 'Nombre del curso (MINUSC)': {172: 'Trabajo en alturas operativo',
  174: 'Trabajo seguro en alturas básico operativo',
  171: 'Trabajo en alturas avanzado',
  27: 'Avanzado trabajo seguro en alturas',
  170: 'Trabajo en alturas administrativo',
  159: 'Reentrenamiento nivel avanzado trabajo seguro en alturas',
  88: 'Fundamentos de redes de datos',
  54: 'Cuentas contables',
  61: 'Diseño asistido por computador - autocad 2d',
  60: '

In [33]:
DF_con.to_csv('Consolidado_Cursos_Sena_2022.csv',index=False)